In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import xarray as xr
import workhorse as wh
import sentinnel_tools as st

# IMPORT DATA

In [2]:
# First deployment (continuous 1-min ping intervals)
adcp01 = st.WinADCP_to_dataset('ADCP/ADCP01_all.txt')

Opening file and extracting data...
 
Creating Pandas dataframe...
 
Dataframe complete
 
Beginning Quality Control Tests
 
Starting ADCP system requirements tests...
Could not run BIT test: No "BIT" column
After Bat: 30058
After Pit: 30057
After Rol: 30040
ADCP system requirements tests complete
After Heading: 30040
 
Starting Beam Quality tests...
Correllation test complete
Percent Good test complete
30038
 
Starting Current Magnitude/Direction tests...
Current Direction test complete
Current Magnitude test complete
30038
 
Starting Component Velocity tests...
Error Velocity test complete
Component Velocity test complete
30038
 
Starting Velocity Shearing and Gradient tests...
Horizontal velocity shear test complete
Vertical velocity shear test complete
Horizontal Current Gradient test complete
30038
 
Starting Echo Intensity test... (This one takes the longest)
Currently on row: 0
Currently on row: 10000
Currently on row: 20000
Currently on row: 30000
Echo Intensity test complete
 


In [3]:
adcp01

<xarray.Dataset>
Dimensions:      (BinDist: 47, bin: 47, time: 24243)
Coordinates:
    BinDist      (bin) float64 0.81 1.06 1.31 1.56 ... 11.56 11.81 12.06 12.31
  * time         (time) datetime64[ns] 2022-07-21T15:05:00 ... 2022-08-08T07:...
Dimensions without coordinates: bin
Data variables:
    Pitch        (time) float64 -1.17 1.85 1.57 1.51 ... -0.12 -0.11 -0.11 -0.1
    Roll         (time) float64 -0.92 0.91 2.93 2.92 ... 1.52 1.53 1.51 1.53
    Heading      (time) float64 38.09 56.64 90.48 90.76 ... 89.49 89.49 89.23
    Temperature  (time) float64 15.15 12.1 12.1 12.06 ... 12.99 12.84 12.52
    Depth        (time) float64 9.526 10.35 10.55 10.56 ... 9.94 9.979 10.18
    East         (time, BinDist) float64 0.008 0.03 0.063 ... 0.016 nan nan
    North        (time, BinDist) float64 0.001 0.041 0.02 ... 0.043 nan nan
    Vertical     (time, BinDist) float64 0.033 0.03 0.036 0.04 ... 0.004 nan nan
    Magnitude    (time, BinDist) float64 0.008 0.051 0.066 ... 0.046 nan nan
    Direction    (time, BinDist) float64 82.9 36.2 72.4 ... 20.4 129.3 81.3
    Correlation  (time, BinDist) int64 122 120 124 120 126 ... 118 112 126 115
Attributes:
    description:          Velocity in beam coordinates
    PingsPerEns:          70
    TimePerPing:          00:00.85
    First Ensemble Date:  22/07/21
    First Ensemble Time:  10:30:00.00
    Ensemble Interval:    60.0
    1st Bin Range:        0.81
    Bin Size:             0.25
    RDI binary file:      C:\Users\tconnolly\Desktop\LG_SWC_Depl\ADCP\SWC0100...
    Instrument:           Broadband 1228.8 kHz

In [6]:
adcp01_1min = adcp01.sel(time=slice('2022-07-21T15:05:00.000000000', '2022-08-08T07:11:00.000000000'))
#adcp01.Time

In [7]:
adcp01_1min

<xarray.Dataset>
Dimensions:      (BinDist: 47, bin: 47, time: 24243)
Coordinates:
    BinDist      (bin) float64 0.81 1.06 1.31 1.56 ... 11.56 11.81 12.06 12.31
  * time         (time) datetime64[ns] 2022-07-21T15:05:00 ... 2022-08-08T07:...
Dimensions without coordinates: bin
Data variables:
    Pitch        (time) float64 -1.17 1.85 1.57 1.51 ... -0.12 -0.11 -0.11 -0.1
    Roll         (time) float64 -0.92 0.91 2.93 2.92 ... 1.52 1.53 1.51 1.53
    Heading      (time) float64 38.09 56.64 90.48 90.76 ... 89.49 89.49 89.23
    Temperature  (time) float64 15.15 12.1 12.1 12.06 ... 12.99 12.84 12.52
    Depth        (time) float64 9.526 10.35 10.55 10.56 ... 9.94 9.979 10.18
    East         (time, BinDist) float64 0.008 0.03 0.063 ... 0.016 nan nan
    North        (time, BinDist) float64 0.001 0.041 0.02 ... 0.043 nan nan
    Vertical     (time, BinDist) float64 0.033 0.03 0.036 0.04 ... 0.004 nan nan
    Magnitude    (time, BinDist) float64 0.008 0.051 0.066 ... 0.046 nan nan
    Direction    (time, BinDist) float64 82.9 36.2 72.4 ... 20.4 129.3 81.3
    Correlation  (time, BinDist) int64 122 120 124 120 126 ... 118 112 126 115
Attributes:
    description:          Velocity in beam coordinates
    PingsPerEns:          70
    TimePerPing:          00:00.85
    First Ensemble Date:  22/07/21
    First Ensemble Time:  10:30:00.00
    Ensemble Interval:    60.0
    1st Bin Range:        0.81
    Bin Size:             0.25
    RDI binary file:      C:\Users\tconnolly\Desktop\LG_SWC_Depl\ADCP\SWC0100...
    Instrument:           Broadband 1228.8 kHz

In [2]:
# Second deployment (1-second ping intervals with 12 sub-pings/second, 20-minute bursts every hour)
adcp02 = st.WinADCP_to_dataset('ADCP/ADCP02_all.txt')

Opening file and extracting data...
 
Creating Pandas dataframe...
 
Dataframe complete
 
Beginning Quality Control Tests
 
Starting ADCP system requirements tests...
After BIT: 210000
After Bat: 202476
After Pit: 202473
After Rol: 202461
ADCP system requirements tests complete
After Heading: 202461
 
Starting Beam Quality tests...
Correllation test complete
Percent Good test complete
202461
 
Starting Current Magnitude/Direction tests...
Current Direction test complete
Current Magnitude test complete
202461
 
Starting Component Velocity tests...
Error Velocity test complete
Component Velocity test complete
202453
 
Starting Velocity Shearing and Gradient tests...
Horizontal velocity shear test complete
Vertical velocity shear test complete
Horizontal Current Gradient test complete
202453
 
Starting Echo Intensity test... (This one takes the longest)
Currently on row: 0
Currently on row: 10000
Currently on row: 20000
Currently on row: 30000
Currently on row: 40000
Currently on row: 500

In [3]:
adcp02

<xarray.Dataset>
Dimensions:      (BinDist: 47, bin: 47, time: 202453)
Coordinates:
    BinDist      (bin) float64 0.81 1.06 1.31 1.56 ... 11.56 11.81 12.06 12.31
  * time         (time) datetime64[ns] 2022-08-23T11:40:00.100000 ... 2022-08...
Dimensions without coordinates: bin
Data variables:
    Pitch        (time) float64 1.2 1.51 1.48 1.49 ... -0.35 1.08 -0.98 -1.95
    Roll         (time) float64 -1.74 -1.48 -1.46 -1.46 ... 4.76 3.06 2.04 2.6
    Heading      (time) float64 281.5 279.8 279.6 279.9 ... 210.4 206.1 204.6
    Temperature  (time) float64 12.46 12.45 12.49 12.46 ... 14.61 14.62 14.62
    Depth        (time) float64 10.46 10.47 10.46 10.46 ... 0.054 0.023 0.087
    East         (time, BinDist) float64 nan nan nan nan ... nan nan nan nan
    North        (time, BinDist) float64 nan nan nan nan ... nan nan nan nan
    Vertical     (time, BinDist) float64 nan nan nan nan ... nan nan nan nan
    Magnitude    (time, BinDist) float64 nan nan nan nan ... nan nan nan nan
    Direction    (time, BinDist) float64 nan 332.1 nan nan ... nan nan nan nan
    Correlation  (time, BinDist) int64 104 124 118 112 99 102 ... 41 50 51 53 56
Attributes:
    description:          Velocity in beam coordinates
    PingsPerEns:          1
    TimePerPing:          00:01.00
    First Ensemble Date:  22/08/23
    First Ensemble Time:  09:40:00.00
    Ensemble Interval:    2.99
    1st Bin Range:        0.81
    Bin Size:             0.25
    RDI binary file:      C:\Users\physoce\Desktop\Logan's_data\SWC2_000.000
    Instrument:           Broadband 1228.8 kHz

In [4]:
# Export the dataset as an netcdf
#adcp01.to_netcdf('ADCP/adcp01.nc')
# Export the dataset as an netcdf
adcp02.to_netcdf('ADCP/adcp02.nc')

# BEAM COORDINATES

In [ ]:
# Import the beam coordinate csv's
b1 = pd.read_csv('ADCP/SWC2_000_1_export.csv')
b2 = pd.read_csv('ADCP/SWC2_000_2_export.csv')
b3 = pd.read_csv('ADCP/SWC2_000_3_export.csv')
b4 = pd.read_csv('ADCP/SWC2_000_4_export.csv')

In [ ]:
# Combine the csv's in chronological order
#data = b1.append(b2)
#data = data.append(b3)
#data = data.append(b4)
#data = data.reset_index(drop=True)
#data = data.drop(columns=['index'])
#data.to_csv('ADCP/beam_data.csv', index=False)
data = pd.read_csv('ADCP/beam_data.csv')

In [ ]:
vel = data.filter(like='V')

In [ ]:
vel_B1 = vel.filter(like='B1')
vel_B2 = vel.filter(like='B2')
vel_B3 = vel.filter(like='B3')
vel_B4 = vel.filter(like='B4')

In [ ]:
time = pd.to_datetime(data.iloc[:,1])
dcell = range(1,48,1)

In [ ]:
B1_arr = vel_B1.to_numpy()
B2_arr = vel_B2.to_numpy()
B3_arr = vel_B3.to_numpy()
B4_arr = vel_B4.to_numpy()

In [ ]:
# create coords
rows = range(1,48,1) 
cols = time

# put data into a dataset
ds = xr.Dataset(
    data_vars=dict(
        B1=(["time", "dcell"], B1_arr),
        B2=(["time", "dcell"], B2_arr),
        B3=(["time", "dcell"], B3_arr),
        B4=(["time", "dcell"], B4_arr)
    ),
    coords=dict(
        dcell=(["dcell"], rows),
        time=(["time"], cols),
    ),
    attrs=dict(description="Velocity in beam coordinates"),
)

In [ ]:
ds['B1'].attrs['units'] = 'mm/s'
ds['B2'].attrs['units'] = 'mm/s'
ds['B3'].attrs['units'] = 'mm/s'
ds['B4'].attrs['units'] = 'mm/s'

In [ ]:
ds